# Get kaggle data

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

In [ ]:
DATA_HOME=r"/content/gdrive/My\ Drive/ml-practico/data"

In [ ]:
!ls $DATA_HOME # revisamos que NO este vacio

In [ ]:
!wget https://machine-learning-practico.s3.amazonaws.com/kaggle_dataset.tgz -O $DATA_HOME/kaggle_dataset.tgz
!tar -C $DATA_HOME -xf $DATA_HOME/kaggle_dataset.tar.gz

# Simple count vectorizer

In [1]:
import sys
sys.path.append('/Users/przivic/prog/machine_learning_practico')

In [19]:
import pandas as pd
df = pd.read_csv('/Users/przivic/Downloads/kaggle_dataset/AllMoviesDetailsCleaned.csv', delimiter=';')

/Users/przivic/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df[df.imdb_id == 'tt0120338'][['original_title', 'overview']].iloc[0].to_dict()

{'original_title': 'Titanic',
 'overview': '84 years later, a 101-year-old woman named Rose DeWitt Bukater tells the story to her granddaughter Lizzy Calvert, Brock Lovett, Lewis Bodine, Bobby Buell and Anatoly Mikailavich on the Keldysh about her life set in April 10th 1912, on a ship called Titanic when young Rose boards the departing ship with the upper-class passengers and her mother, Ruth DeWitt Bukater, and her fiancé, Caledon Hockley. Meanwhile, a drifter and artist named Jack Dawson and his best friend Fabrizio De Rossi win third-class tickets to the ship in a game. And she explains the whole story from departure until the death of Titanic on its first and last voyage April 15th, 1912 at 2:20 in the morning.'}

In [4]:
from collections import Counter
import re
wp = re.compile('\w+')
s = Counter()

In [5]:
for t in df.overview.dropna():
    for tt in wp.findall(t.lower()):
        s[tt] += 1

In [6]:
sorted(s.items(), key=lambda x: -x[1])[:10]

[('the', 831347),
 ('a', 517532),
 ('and', 441625),
 ('of', 423157),
 ('to', 369825),
 ('in', 284461),
 ('is', 195049),
 ('his', 167710),
 ('s', 144823),
 ('with', 129446)]

In [7]:
sorted(s.items(), key=lambda x: x[1])[:10]

[('taisto', 1),
 ('kasurinen', 1),
 ('nikander', 1),
 ('2257', 1),
 ('hynson', 1),
 ('swafford', 1),
 ('earger', 1),
 ('himmelman', 1),
 ('icebound', 1),
 ('immensities', 1)]

In [8]:
len(s)

244391

In [67]:
from dateutil.parser import parse

df = df[~df.release_date.isna()].copy()
df['release_year'] = df.release_date.apply(lambda x: parse(x).year)
df = df[df.release_year > 1970].copy()
df = df[df.vote_average > 0].copy()
df = df[~df.overview.isna()].copy()

len(df)

93787

In [51]:
train = df[df.release_year <= 2015]
test = df[df.release_year > 2015]

len(test) / len(train), len(test)

(0.06719237158917639, 5905)

In [52]:
X_train = train.overview.tolist()
X_test = test.overview.tolist()

y_train = train.vote_average > 7.5
y_test = test.vote_average > 7.5

In [53]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import roc_auc_score

In [58]:
pipe = make_pipeline(CountVectorizer(), LogisticRegression())
pipe.fit(X_train, y_train)

print('train', roc_auc_score(y_train, pipe.predict_proba(X_train)[:, 1]))
print('test', roc_auc_score(y_test, pipe.predict_proba(X_test)[:, 1]))

/Users/przivic/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:760: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


train 0.9073028834189234
test 0.593222697392902


In [ ]:
pipe = make_pipeline(TfidfVectorizer(), LogisticRegression())
pipe.fit(X_train, y_train)

print('train', roc_auc_score(y_train, pipe.predict_proba(X_train)[:, 1]))
print('test', roc_auc_score(y_test, pipe.predict_proba(X_test)[:, 1]))

In [62]:
pipe = make_pipeline(CountVectorizer(min_df=20), LogisticRegression())
pipe.fit(X_train, y_train)

print('train', roc_auc_score(y_train, pipe.predict_proba(X_train)[:, 1]))
print('test', roc_auc_score(y_test, pipe.predict_proba(X_test)[:, 1]))

/Users/przivic/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:760: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


train 0.8245471339506503
test 0.6066093214867277


In [63]:
pipe = make_pipeline(CountVectorizer(min_df=20, ngram_range=(1, 3)), LogisticRegression())
pipe.fit(X_train, y_train)

print('train', roc_auc_score(y_train, pipe.predict_proba(X_train)[:, 1]))
print('test', roc_auc_score(y_test, pipe.predict_proba(X_test)[:, 1]))

/Users/przivic/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:760: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


train 0.9333554357385985
test 0.5940832917403344


In [65]:
pipe = make_pipeline(TfidfVectorizer(), LogisticRegression())
pipe.fit(X_train, y_train)

print('train', roc_auc_score(y_train, pipe.predict_proba(X_train)[:, 1]))
print('test', roc_auc_score(y_test, pipe.predict_proba(X_test)[:, 1]))

/Users/przivic/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:760: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


train 0.8501446935200785
test 0.6478523358024775


In [66]:
pipe = make_pipeline(TfidfVectorizer(ngram_range=(1,3)), LogisticRegression())
pipe.fit(X_train, y_train)

print('train', roc_auc_score(y_train, pipe.predict_proba(X_train)[:, 1]))
print('test', roc_auc_score(y_test, pipe.predict_proba(X_test)[:, 1]))

train 0.9647089264333337
test 0.6566005144026713
